In [ ]:
import h5py
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from sklearn.metrics import roc_curve, auc
import tensorflow.keras.backend as K
import joblib

# compute a mask per event, and attach vec(0) for the first MLP output for those masked
# so S and pho steps are unchanged, but skipping first MLP computations
# the total dim N * D is still the same after the first MLP (those zero constituents are skipping phi)
# --> for zero-padded constituents, directly make D-dim zeros for them after the first MLP
# make a route:
# if mask == 0:
#     pass to first MLP
# else:
#     make D-dim zeros

In [ ]:
'''
['jetConstituentList',
 'jetFeatureNames', 59 (g,q,w,z,t = -6:-1)
 'jetImage',
 'jetImageECAL',
 'jetImageHCAL',
 'jets', 59
 'particleFeatureNames'] 17 (pt = 5, etarel = 8, phirel = 11)
'''
h5_files = glob.glob('val/jetImage*.h5') + glob.glob('train/jetImage*.h5')
x = []
y = []

for h5_file in h5_files:
    with h5py.File(h5_file, 'r') as f:
        x.append(f['jetConstituentList'][:, :, [5, 8, 11]])
        y.append(f['jets'][:, -6:-1])
        
x = np.concatenate(x, axis = 0)
y = np.concatenate(y, axis = 0)

with h5py.File('data.h5', 'w') as f:
    f.create_dataset('x', data=x)
    f.create_dataset('y', data=y)

In [ ]:
with h5py.File('data.h5', 'r') as f:
    x = f['x'][:, :32, :]
    y = f['y'][:]
    
x[x[:, :, 0] < 2] = 0

q5 = np.percentile(x[:, :, 0], 5)
q95 = np.percentile(x[:, :, 0], 95)
x[:, :, 0] = (x[:, :, 0] - q5) / (q95 - q5)

train_ratio = 0.6
val_ratio = 0.2
test_ratio = 1 - train_ratio - val_ratio
X_train_val, X_test = train_test_split(x, test_size = test_ratio, random_state = 42)
X_train, X_val = train_test_split(X_train_val, test_size = val_ratio/(val_ratio + train_ratio), random_state = 42)
print('X_train shape: ' + str(X_train.shape))
print('X_val   shape: ' + str(X_val.shape))
print('X_test  shape: ' + str(X_test.shape))
del X_train_val